# Basics of Mobile robotics - Project report

### Students: Sushen Jilla Venkatesa 284402, Forero Philippe 284213, Mael Mouhoub 288287, Selina Bothner 284028

## 1.Introduction

We have realized this project in the framework of the course Basics of mobile robotics at EPFL in 2021. The goal of this project is to offer students the possibility to learn the basics of robotics, building a complex project within 4 weeks. We used a Thymio robot [1] and a webcam for this project. Our main constraints for this project were to integrate:

- Computer vision: Take a picture and do image processing to detect all elements of the map.
-	Motion control:  A Finite State Machine (FSM) with different modes, controlling the robot motors
-	Global and Local navigation: Find the shortest path to the goal, and avoids obstacles placed randomly while moving towards the goal 
-	Filtering: diverse filters in Computer vision and Kalman filter in Motion control


 We will combine these elements into a global project that will allow us to maneuver the Thymio robot in a defined map around obstacles. For more details, we will discuss the challenges linked to each part in the following sub-sections.

## 2. The environment

We have created a special environment to run the project.

![Map_real_1](images/Map_real_1.jpg)

This photo is taken with the webcam we use, and we can see here everything there is on the environment. We have covered the ground with cardboard to avoid ground pattern and false map border detection. To make the system as robust as possible, we also used a carboard on top of the webcam to avoid shadows from the overhead lights, as well as led lights to illuminate the map. This allows for the color detection used by the camera to be more robust as the lighting will always be about the same. We will go into more details of the choices of the colors for each item present on the map in the Vision module.

## 3. Vision

The camera part uses mainly the OpenCV library.<br/><br/>
The program starts off with the camera. It takes a picture and analyzes it. After every picture and every analysis, the user is asked if it seems okay. This is done to ensure a good map initialization, because sometimes the elements are not well detected. These issues arises if light conditions are not satisfied, or that the map is being out of bounds of camera field of view, etc.. <br/><br/>
The camera part is divided in several parts: <br/>
The outer border detection uses a picture taken by the camera, on which we apply a Canny filter[2]. This filter works by finding the region in an image where there is a strong change in intensity. This allows to detect contours in the image. We then sort through the different contours and keep the biggest one as this represents the border of the map. <br/>
We must then crop this image around the outer contour. We perform a crop on the image and a rotation in the following way: <br/>

![MapTurn](images/MapTurn.jpg)
![ImgMapTurn](images/ImgMapTurn.jpg)

This method was inspired by an example [3].<br/><br/>
To make the algorithm work, it is imperative that that the ‘A’ corner is the one that is at the bottom of the image. If this is not the case the image will be rotated the other way. This is because we use a function from OpenCV that sets the lowest point as the first one, and then goes in a clockwise direction. As these points are then used to rotate the image, if they are not in the right order, the image will then be rotated in the other direction.<br/>
Now that we have a reshaped image, we can start detecting the various elements we need to make the program work. Each of these elements are color coded, and all work relatively similarly. We used red to detect the obstacles, blue to detect the goal point and green to detect Thymio. We chose these colors as they are on opposite side of the spectrum from each other. <br/>
In order to separate these colors, we use color masks, coded in HSV [4]. <br/>
To detect the obstacles, we simply look for contours in the red mask. Noise and errors can be reduced by limiting the size of the obstacles detected. <br/>
To detect the goal, we do the same thing but with the color blue. Once the goal is detected, we simply take the center of the rectangle as the goal point that will be used for the global navigation.
The detection of the Thymio is also similar. The only difference, apart from the color is the fact that we have two rectangles to detect. This is done so that we can also determine the orientation of the robot. The center of the smallest rectangle (in the back) determines the position of the Thymio (the middle point between the wheels). We use the center of both rectangles to determine the angle of the robot. <br/>
Once all these elements are detected, take the contours of the obstacles, and put them in a matrix that can be used by the global navigation. To put the obstacles into the matrix, we use a certain simplification. We find the four outer points and use them to create a square. This is not necessarily the most precise, but it is not problematic because we must anyways make the obstacles bigger, in order to take into account, the size of the Thymio.<br/> 



![obstacleMatrix](images/obstacleMatrix.jpg)
![BoxPoints](images/BoxPoints.jpg)

To do these tasks, we used several examples and tutorials found on internet, such as this video [5].

## 4.Global navigation

The global navigation starts after the analysis of the objects by the vision module. We receive from the vision module a matrix (containing ‘1s’ or ‘0s’) defining if the pixel is an obstacle or not. We also receive the start and goal points. For global navigation, we chose the A* algorithm. Take a look at a typical result of the A*star algorithm in our program.

![Astar_result_video](images/Astar_result_video.png)

In this algorithm we will find the shortest path to goal, and we will memorize in memory the total path and the checkpoints of the path. The checkpoints are points in path that are turning points, only theses points are important, as they are going to be passed to motor control. The starting angle of the Thymio will also be passed. To take into account the real size of the Thymio and of the obstacles, the algorithm has scaling factor memorized (the size of a grid point in milimeters). Note that: We also made the obstacles bigger to take into account the size of the Thymio.<br/><br/>
The choice of the A* algorithm is mainly due to its completeness and optimality. It is well suited for this project as the camera provide a discrete map and its easy to implement. Others global navigation methods were presented in the course but were not chosen for several reasons. Visibility maps or cell decomposition maps are quite complex and will not give better results for this project. Potential fields maps are easy to understand but at a cost that the robot can be trapped in a local minima and never reach the goal. One major drawback of the A* is that it’s quite computationally heavy for very large maps and tends to be very slow.<br/><br/> 
Several timing testing of the A* algorithm gave a good estimate of maximum size of map (50x50) for fast running algorithms. 

Here's a recap of some of the timer tests:

| dimensions  | Map without growth obstacles[s] | Map with growth Obstacles[s] |
|:---------------------------:|:---------------------:|:-------------------------------:|
|            50x50            |           2.25        |           1.87          |
|           100x100           |         43.3         |           59.48           |

Complexity increase dramatically with the size of the map, the cost of the main function in time order for a 100x100:
1. A star algorithm (54s)
2. Calculating checkpoints(3.88s)
3. Plotting(1.43s) 
4. Growing the obstacles(0.17s)

Knowing that, all elements from the camera analysis must then be downsized to maximum (50x50) in order to get a fast enough A* algorithm. 

## 5.Control law

The control law of the Thymio can be split into five states whose simplified operation is given in the Finite State Machine below. <br/><br/>
**States**: <br/>
Starting : The robot waits until it is detected by the camera and as soon as this is the case, it initializes the first checkpoint as the current checkpoint aim of the robot. <br/>
Afterwards, it switches to the Rotating state in order to angle itself in a proper manner to get to the current checkpoint aimed at. <br/> <br/>
Rotating : The robot rotates until it aligns with the direction of the target checkpoint it wants to reach. In order to have an overall smoother motion, we leave a margin of error using a threshold such that when the difference between the estimated angle and the wanted angle is lower than this value, then we switch to the Moving state. <br/> <br/>
Moving : The robot advances in a straight line towards the checkpoint it is aiming at (current checkpoint), we make use of a distance threshold such that we increment to the next checkpoint if the distance between the thymio and the current checkpoint is below the value. <br/> 
If it has deviated from its direction it returns to the Rotating state. <br/>
As soon as it has arrived at the target checkpoint, two cases are to be differentiated : <br/>
-	It is not the last checkpoint, then it changes its target checkpoint (increments the new_checkpoint_index by one) and returns to the Rotating state <br/> 
-	It is the last checkpoint, then it goes to Stop state  <br/> <br/>
Stop : The Stop state stops the robot when the end goal has been reached. We also set a bool to true which will allow us to end the timer running the program. <br/> <br/>
Obstacle avoidance : If an obstacle is detected by one of the horizontal proximity sensors of the robot, it starts to rotate in the clockwise direction until none of the sensors detect any obstacle, we also make use of a threshold here. <br/> 
It then returns to the rotating state which will try to direct it to the next checkpoint and so on until it has passed the obstacle (see figure) <br/> <br/>


![Obstacle_avoidance.drawio](images/Obstacle_avoidance.png)

### Inputs and outputs
The control law needs 5 inputs : <br/> <br/>
sensor_data: array containing the values of the 5 front horizontal proximity sensors <br/> <br/>
checkpoints_data: set/array containing all the checkpoints the thymio has to go through, each row is a checkpoint <br/> <br/>
fsm_state: indicates what state of the fsm we are in ("starting", "moving", "rotating", "obstacle_avoidance", "stop") <br/> <br/>
state_kalman: state estimate from kalman <br/> <br/> <br/> 
And return 4 output : <br/> <br/>
left_velocities and right_velocities  : velocity of left/right wheel as decided by control law in metric [mm/s] <br/> <br/> 
new_fsm_state: updated fsm_state <br/> <br/>
new_checkpoint_index: updated checkpoint index <br/>

![Control_Law](images/Control_Law.png)

## 6.Kalman filter

For the extended Kalman filter, we first needed to model the way the Thymio moves with respect to it’s motor speeds. Looking at the image below, we see how we reference the position and angle of the Thymio. <br/>


![KalmanFilter](images/KalmanFilter.png)

We end up with a nonlinear function to update the position and state as can be seen in the equation below :

![Kalman](images/Kalman.jpg)

Then following the usual extended Kalman filter algorithm, we implement it in Python. <br/>
Now the way it works is that whenever the camera is available and is able to find the Thymio on the map, then we make use of that as a measurement for the correction step. Otherwise when the camera isn’t available we only use the update step using the odometry. <br/>
We have also added some protective measures to use directly the camera estimation instead when we have huge spikes in the estimations which wouldn’t make sense. <br/>


## 7.Conclusion

This project has been a great way for us to apply the different concepts seen in the course that it is the vision, the global and local navigation as well as the uncertainty management (Kalman filter). It has also been the opportunity to see the importance and the difficulty of the pooling of independently developed modules even when they are jointly thought upstream. <br/><br/> 
To conclude, even if we had some difficulties with the Kalman filter or the camera, we think we have succeeded in making a project that shows the different concepts of the course and fits in our project specifications

## 8. References

[1: https://www.thymio.org/fr/] <br/>
[2 : https://en.wikipedia.org/wiki/Canny_edge_detector] <br/>
[3 : https://jdhao.github.io/2019/02/23/crop_rotated_rectangle_opencv] <br/>
[4 : https://fr.wikipedia.org/wiki/HSV] <br/>
[5 : https://www.youtube.com/watch?v=tk9war7_y0Q] <br/>
[6 : https://en.wikipedia.org/wiki/Extended_Kalman_filter]<br/>
[7 : https://moodle.epfl.ch/mod/resource/view.php?id=1090142]<br/>
[8 : https://docs.opencv.org/3.4/d2/d96/tutorial_py_table_of_contents_imgproc.html]

## 9. Test run of the whole project

In [ ]:
%run src\project\Global_nav.py
%run src\project\DetectionFcts.py
%run src\project\Locomotion.py

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
def main(checkpoints_data,m, contours, area_max, Q, R, Ts, webcam) :
    global current_checkpoint_index, fsm_state, cam_available, cam_data, x_est_prev, P_est_prev, reached_end, speed
    """
    Main function which coordinates the whole application
    
    global : takes the variables that will be changed throughout the application
    
    param checkpoints_data: velocity of left wheel as decided by control law in metric [mm/s]
    param Q: Q matrix for the Kalman Filter
    param R: R matrix for the Kalman Filter
    param Ts: Period for the Python thread which uses the main(...) function
    
    no return 
    """
    if webcam:
        success,img = cap.read()
        cam_available = True
    else: 
        cam_available = False
    
    if cam_available:
        Thymio_coord, Thymio_found = analyse_thymio(img,contours, area_max, m.ratio_downscale)
    
    if (Thymio_found):
        corrected_estimate_angle = -Thymio_coord[2]
        if corrected_estimate_angle < 0 :
            corrected_estimate_angle = corrected_estimate_angle +2*math.pi
        Thymio_coord = (Thymio_coord[0]*m.ratio_total, (m.height-Thymio_coord[1])*m.ratio_total, corrected_estimate_angle)
    else:
        Thymio_coord = np.array([0,0,0])
    [x_est_prev, P_est_prev] = kalman_filter(speed, Thymio_found, Thymio_coord, x_est_prev, P_est_prev, Q, R, Ts)
    
    sensor_data = np.array(node.v.prox.horizontal[0:5]) #get data from horiz proximity sensors
    
    [left_velocity, right_velocity, fsm_state, current_checkpoint_index, reached_end] = control_law(sensor_data, checkpoints_data, current_checkpoint_index, fsm_state, x_est_prev, Thymio_found)
    print('Checkpoint number : '+str(current_checkpoint_index) + '\n')
    print('FSM state : '+str(fsm_state) + '\n \n')
    
    speed = [left_velocity, right_velocity]
    
    node.send_set_variables(motors(left_velocity, right_velocity))
    cam_avaiable = False 
    Thymio_found = False

In [ ]:
import os
import sys
import math
from statistics import mean
import numpy as np
import cv2
# import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
%matplotlib inline


# Camera intialisation
webcam = True
cap = cv2.VideoCapture(1)
cap.open(1, cv2.CAP_DSHOW)

# While loop to get a proper first image
while True:
    if webcam:
        success,img = cap.read()
        plt.figure(figsize=(7, 7))
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb_img)
        plt.show()
        
    keypress1 = input("Satisfying Image ? : ")
    if keypress1 =="y":
        m, contours, area_max = analyse(img)
        keypress2 = input("Satisfying analysis ? : ")
        if keypress2 =="y":
            break

### Control Law parameters intialization
checkpoints_data = m.checkpoints.transpose()*m.ratio_total # checkpoints converted to mm
length_checkpoint = np.size(checkpoints_data, 0)
for i in range(length_checkpoint):
    # adjusting checkpoints due to different references
    checkpoints_data[i][1] = m.height*m.ratio_total - checkpoints_data[i][1] 

current_checkpoint_index = 0
fsm_state = "starting"
left_velocity = 0
right_velocity = 0 
speed = [0, 0]

### Kalman filter parameters initalization
corrected_start_angle = -m.start[2]
if corrected_start_angle < 0 :
    corrected_start_angle = corrected_start_angle + 2*math.pi
    
x_init = np.array([m.start[0]*m.ratio_total, (m.height-m.start[1])*m.ratio_total, corrected_start_angle])
P_init = np.diag([0.45, 0.45, 0.4])
x_est_prev = x_init 
P_est_prev = P_init

Q = np.diag([0.3, 0.3, 0.5])                  
R = np.diag([0.7, 0.7, 0.9])

### End of application bool
reached_end = False

### Thread parameter
Ts = 0.2

timer = RepeatedTimer(Ts, main, checkpoints_data,m, contours, area_max, Q, R, Ts, webcam) # it auto-starts, no need of rt.start()   

try:
    while not(reached_end) : 
        await node.wait_for_variables() # wait for Thymio variables values
        await client.sleep(1)
finally:
    timer.stop() 
    print('Application has finished, bye bye !')


In [ ]:
#Now unlock the robot:
await node.unlock()